In [ ]:
from dolfin import *
import matplotlib.pyplot as plt
import numpy as np
import solver as sol


## 1D Model


In [ ]:
mesh = UnitSquareMesh(20,20)
m0 = 1e-3
dt = 0.5
T = 150
save_interval = 2
times = []#[0,7/dt,14/dt,21/dt,28/dt]
doses = []#10*np.ones_like(times)
V = FunctionSpace(mesh,"P",2)
n0 = Expression("m0/(pow(2*pi,0.5)*sigma)*exp(-pow(x[1]-s0,2)/(2*sigma*sigma))",m0 = m0,s0 = 0.5,sigma=sqrt(0.02),degree=2)
n0 = interpolate(n0,V)
c0 = interpolate(Constant(1.0), V)
f = Constant(0.0)
path_sol = "solutions/test1D"

solver = sol.Solver1D(mesh, V, n0, c0, dt, T, save_interval, times, doses, path_sol, f)
# solver.set_parameters({'K_m':0.005})
solver.solve()

In [ ]:
mass = np.load('solutions/test1D/mass.npy')
plt.plot(mass)

In [ ]:
mesh = UnitSquareMesh(20,20)
dt = 0.1
T = 10
save_interval = 10
path_init = '../Model1D/solutions/B_Km1e-2' # path to the initial conditions
c0file = XDMFFile(f'{path_init}/c.xdmf')
n0file = XDMFFile(f'{path_init}/n.xdmf')
V = FunctionSpace(mesh,"P",2)
c0 = Function(V)
n0 = Function(V)
c0file.read_checkpoint(c0,"c",100)
n0file.read_checkpoint(n0,"n",100)
times = [1,8,15,22,29]
doses = 10*np.ones_like(times)
f = Constant(0.0)

path_sol = "solutions/test_uhfrt"

solver = sol.Solver1D(mesh, V, n0, c0, dt, T, save_interval, times, doses, path_sol,f)
solver.bdf2()

## 2D Model

In [ ]:
mesh = BoxMesh(Point(-1, -1, 0), Point(1, 1, 1), 20, 20, 10)
m0 = 1e-3
dt = 0.5
T = 5
save_interval = 2
times = [] #[0,7/dt,14/dt,21/dt,28/dt]
doses = [] #10*np.ones_like(times)  
V = FunctionSpace(mesh,"P",2)
n0 = Expression("m0/(pow(2*pi,0.5)*sigma)*exp(-pow(x[2]-s0,2)/(2*sigma*sigma) - x[0]*x[0]/(2*sigma_x*sigma_x) - x[1]*x[1]/(2*sigma_x*sigma_x))",
                m0 = m0,s0 = 0.5,sigma=sqrt(0.02),sigma_x = sqrt(0.008),degree=2)
n0 = interpolate(n0,V)
c0 = interpolate(Constant(1.0), V)
f = Expression('3*1e5*exp(-x[0]*x[0]/(sigma_v*sigma_v) - x[1]*x[1]/(sigma_v*sigma_v))',sigma_v = 0.1,degree=2)
path_sol = "solutions/test2D2"

solver = sol.Solver2D(mesh, V, n0, c0, dt, T, save_interval, times, doses, path_sol, f, sym=False)
solver.solve()